In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import sys
from tensorflow.keras.utils import to_categorical


2025-03-07 10:42:42.567781: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-07 10:42:42.575826: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-07 10:42:42.603871: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741365762.649206  100626 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741365762.664500  100626 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

def create_vit_model(input_shape, num_classes, patch_size=10, projection_dim=64, num_heads=4, transformer_units=[64, 32], transformer_layers=4, mlp_head_units=[128, 64]):
    inputs = layers.Input(shape=input_shape)
    resized = layers.Resizing(input_shape[0], input_shape[1])(inputs)
    scaled = layers.Rescaling(1.0 / 255)(resized)
    patches = Patches(patch_size)(scaled) # Correct way to call the layer
    patches = layers.Dense(projection_dim)(patches)

    num_patches = patches.shape[1]
    positions = tf.range(start=0, limit=num_patches, delta=1)
    position_embedding = layers.Embedding(input_dim=num_patches, output_dim=projection_dim)(positions)
    encoded_patches = patches + position_embedding

    for _ in range(transformer_layers):
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)(x1, x1)
        x2 = layers.Add()([attention_output, encoded_patches])
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = layers.Dense(transformer_units[0], activation="relu")(x3)
        x3 = layers.Dense(transformer_units[1])(x3)
        encoded_patches = layers.Add()([x3, x2])

    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    for units in mlp_head_units:
        representation = layers.Dense(units, activation=tf.nn.gelu)(representation)
        representation = layers.Dropout(0.5)(representation)
    outputs = layers.Dense(num_classes, activation="softmax")(representation)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

input_shape = (100, 100, 1)
num_classes = 12
vit_model = create_vit_model(input_shape, num_classes)   

TypeError: Exception encountered when calling Patches.call().

[1mCould not automatically infer the output shape / dtype of 'patches' (of type Patches). Either the `Patches.call()` method is incorrect, or you need to implement the `Patches.compute_output_spec() / compute_output_shape()` method. Error encountered:

Missing required positional argument[0m

Arguments received by Patches.call():
  • args=('<KerasTensor shape=(None, 100, 100, 1), dtype=float32, sparse=False, name=keras_tensor_2>',)
  • kwargs=<class 'inspect._empty'>

In [3]:
input_shape = (100, 100, 1)
num_classes = 12
vit_model = create_vit_model(input_shape, num_classes)

TypeError: Exception encountered when calling Patches.call().

[1mCould not automatically infer the output shape / dtype of 'patches_1' (of type Patches). Either the `Patches.call()` method is incorrect, or you need to implement the `Patches.compute_output_spec() / compute_output_shape()` method. Error encountered:

Missing required positional argument[0m

Arguments received by Patches.call():
  • args=('<KerasTensor shape=(None, 100, 100, 1), dtype=float32, sparse=False, name=keras_tensor_5>',)
  • kwargs=<class 'inspect._empty'>

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class SimpleLayer(layers.Layer):
    def call(self, inputs):
        return inputs * 2

input_tensor = tf.ones((1, 5))
simple_layer = SimpleLayer()
output_tensor = simple_layer(input_tensor)
print(output_tensor)

tf.Tensor([[2. 2. 2. 2. 2.]], shape=(1, 5), dtype=float32)


2025-03-07 10:44:03.389674: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [7]:
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)

2.18.0
3.8.0
